## Imports

In [1]:
import numpy as np

## Self-attention

In [ ]:
def compute_qkv(X, W_q, W_k, W_v):
    # Do dot product of input and weights
	Q = X @ W_q
	K = X @ W_k
	V = X @ W_v
	return Q, K, V

def softmax(X):
    # Doing this make the softmax numerically stable
    X_shifted = X - np.max(X, axis=1, keepdims=True)
    return np.exp(X_shifted)/np.sum(np.exp(X_shifted), axis=1, keepdims=True)

def self_attention(Q, K, V):
    d_k = Q.shape[1]
    scores = (Q @ K.T) / np.sqrt(d_k)
    attention_weights = softmax(scores)
    attention_output = np.matmul(attention_weights, V)
    return attention_output

In [20]:
X = np.array([[1, 0], [0, 1]])
W_q = np.array([[1, 0], [0, 1]])
W_k = np.array([[1, 0], [0, 1]])
W_v = np.array([[1, 2], [3, 4]])


In [21]:
Q, K, V = compute_qkv(X, W_q, W_k, W_v)
output = self_attention(Q, K, V)

In [22]:
print(output)

[[1.6604769 2.6604769]
 [2.3395231 3.3395231]]


## Multi-head attention

In [34]:
def compute_qkv(X, W_q, W_k, W_v):
    Q = X @ W_q
    K = X @ W_k
    V = X @ W_v
    return Q, K, V

def softmax(X):
    X_shifted = X - np.max(X, axis=1, keepdims=True)
    return np.exp(X_shifted) / np.sum(np.exp(X_shifted), axis=1, keepdims=True)

def self_attention(Q, K, V):
    dk = K.shape[1]
    scores = (Q @ K.T)/ np.sqrt(dk)
    attn_wei = softmax(scores)
    attn_outputs = attn_wei @ V
    return attn_outputs


def multi_head_attention(Q, K, V, n_heads):
    seq_len, d_model = X.shape
    # Need to ensure the dimension can be divided equally for all heads
    assert d_model % n_heads == 0

    d_k = d_model // n_heads

    # perform splits over heads
    Q_reshaped = Q.reshape(seq_len, n_heads, d_k).transpose(1,0,2)
    K_reshaped = K.reshape(seq_len, n_heads, d_k).transpose(1,0,2)
    V_reshaped = V.reshape(seq_len, n_heads, d_k).transpose(1,0,2)

    attention = []

    # calculate attention
    for i in range(n_heads):
        attn = self_attention(Q_reshaped[i],K_reshaped[i], V_reshaped[i])
        attention.append(attn)
    
    # concatenate the attention output
    attention_op = np.concatenate(attention, axis=-1)
    return attention_op

In [35]:
m, n = 4, 4 
n_heads = 2 
np.random.seed(42) 
X = np.arange(m*n).reshape(m,n) 
X = np.random.permutation(X.flatten()).reshape(m, n) 

W_q = np.random.randint(0,4,size=(n,n)) 
W_k = np.random.randint(0,5,size=(n,n)) 
W_v = np.random.randint(0,6,size=(n,n)) 

Q, K, V = compute_qkv(X, W_q, W_k, W_v) 
print(multi_head_attention(Q, K, V, n_heads))

[[103. 109.  46.  99.]
 [103. 109.  46.  99.]
 [103. 109.  46.  99.]
 [103. 109.  46.  99.]]


## Masked multi-head attention